
# Investigate a CPI  Dataset 

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> **The Consumer Price Index (CPI)** : 
>
>is a measure of the average change over time in the prices paid by urban consumers for a market basket of consumer goods and services.
>

> **ask questions first** 
>
>Great questions help you focus on relevant parts of your data and direct your analysis towards meaningful insights. 
>

>1- Is the value of CPI in the world increasing over time or decreasing?
>

>2- Does the CPI of Saudi Arabia is higher than the average  CPI of the world over time?
>

>3- Does the Income level affect the CPI?
>

>4- In the last 20 years What are the top ten countries in terms of CPI?
>
>
>

>5- In the last 20 years What are the least ten countries in terms of CPI?
>
>
>



In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
%matplotlib inline

<a id='wrangling'></a>
### Data Wrangling

>  we will load in the data, check for cleanliness, and then trim and clean our dataset for analysis.

### General Properties

 I bring this dataset from the world bank 
> https://data.worldbank.org/indicator/FP.CPI.TOTL
>

In [2]:
# Load data 

df  = pd.read_csv('API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_988671.csv' )
# copy data from df to df_clean 
df_clean = df.copy()

FileNotFoundError: [Errno 2] File API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_988671.csv does not exist: 'API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_988671.csv'

>Perform operations to inspect data 

In [ ]:
#print out a few lines.
df_clean.head()
#from head command we found 
# Messy data, also known as untidy data. Untidy data has structural issues.
# columns are not as useful as  : 1-Country Code 2- Indicator Name	Indicator 3-Indicator Code

In [ ]:
df_clean.shape
# we have 264 rows and 64 columns 

In [ ]:
# check missing value and data type 
df_clean.info()
# I noted From info command we found that there are many years that contain null values
# no issue with data type 

In [ ]:
df_clean.describe()

In [ ]:
# specify more years with null values
df_clean.isnull().sum(axis=0).nlargest(40)
# I noted most missing value from "1960" to "1990" and "2019" 

### Data Cleaning

In [ ]:
# After discussing the structure of the data and any problems that need to be
# I noted : 

# Messy data, also known as untidy data. Untidy data has structural issues.
# Columns are not as useful as  : 1-Country Code 2- Indicator Name	Indicator 3-Indicator Code

# From info command we found that there are many years that contain null values
# most missing value from 1960 to 1990 and 2019 

In [ ]:
# drop coulmns from 1960 to 1990 and 2019 
df_clean.drop(columns=df_clean.iloc[:,4:35] , inplace=True )
 # check of drop columns from 1960 to 1990 
df_clean.columns.to_list()

In [ ]:
# drop all copuntry without any value
df_years = df_clean.iloc[:,4:]
df_years_col = df_years.columns.to_list()
df_clean.dropna(subset = df_years_col, how='all' , inplace=True)

In [ ]:
 # check of drop all copuntry without any value
#number of country before deletion "264"
df_clean.shape[0]
# now 233 so 31 country without any value 

In [ ]:
# drop not useful columns 
df_clean.drop(columns=['Country Code','Indicator Name','Indicator Code'] , inplace=True)
df_clean.head()

In [ ]:
# Restructur DataFrame
formatted_df = pd.melt(df_clean,
                       ["Country Name"],
                       var_name="year",
                       value_name="consumer prices")
formatted_df = formatted_df.sort_values(by=["Country Name"])
formatted_df.head(10)

In [ ]:
df_clean.describe()

In [ ]:
# sorting dataFrame
df_clean = formatted_df.sort_values(['Country Name', 'year'], ascending=True )
df_clean

In [ ]:
# rest index
df_clean.reset_index(drop=True , inplace=True)
df_clean

In [ ]:
# renamed columns to be lower cased and have spaces represented by an underscore 
df_clean.columns = [i.replace(' ', '_').lower() for i in df_clean.columns]

In [ ]:
df_clean.country_name.unique().tolist()

# from this list I found the most useful for us  = 

#Saudi Arabia
#world
#High income
#Middle income
#Low income

In [ ]:
# ckeck data type  and null values
df_clean.info()
# we need conver year to datatime type 

In [ ]:
#conver year to datatime type 
df_clean.year = pd.to_datetime(df_clean.year, format='%Y' )

In [ ]:
# fill null values with mean of the consumer_prices for each country 
df_clean['consumer_prices'] = df_clean['consumer_prices'].fillna(df_clean.groupby('country_name')['consumer_prices'].transform('mean'))

In [ ]:
df_clean

<a id='eda'></a>
## Exploratory Data Analysis

>  After cleaned our data, we're ready to move on to exploration.


#### Q 1 : Is the value of CPI in the world increasing over time or decreasing?

In [ ]:
#world
list_of_year = (pd.DatetimeIndex(df_clean['year']).year).unique()
world = df_clean['consumer_prices'].loc[df_clean.country_name == 'World']


plt.figure(figsize=(20,10))
plt.title('Consumer Price Index (CPI) over Time of the world ', fontdict={'fontweight':'bold', 'fontsize': 25})

plt.plot(list_of_year, world, 'b.-', label='World')

plt.xticks(list_of_year)
plt.xlabel('Year' , fontsize =  25)
plt.ylabel('consumer prices (annual %)' ,fontsize =  25)
plt.legend()

plt.show()

###### Answer  Q 1 : Is the value of CPI in the world increasing over time or decreasing?
>The value of CPI from thirty years is **Decreasing**, except in 2008 there was an unusual change, and after research on the most important events that happened to the world in 2007-2008 i think the main reason for this change  is **financial crisis**



>What happened during the financial crisis?
>
>Referred to as the worst economic disaster since the Great Depression, the 2008 financial crisis devastated the world economy. This resulted in what's known as the Great Recession, which led to falling housing prices and sharp increases in unemployment. The associated repercussions were enormous, and are still influencing financial systems today. 
In the US, more than eight million citizens lost their jobs, approximately 2.5 million businesses were devastated, and close to four million homes were foreclosed in less than two years. From food insecurity to income inequality, many have lost faith in the system.
The recession officially ended in 2009, but many continued to suffer long after it, especially in the US. The unemployment rate reached 10% in 2009 and was only recovered to pre-crisis levels in 2016.>

 sourse : https://www.binance.vision/economics/the-2008-financial-crisis-explained

#### Q 2 : Does the CPI of Saudi Arabia is higher than the average  CPI of the world over time?

In [ ]:
#Saudi Arabia
#world
saudi_arabia = df_clean['consumer_prices'].loc[df_clean.country_name == 'Saudi Arabia']

plt.figure(figsize=(20,10))
plt.title('Consumer Price Index (CPI) over Time ',fontdict={'fontweight':'bold', 'fontsize': 25})

plt.plot(list_of_year, saudi_arabia, 'g.-', label='Saudi Arabia')
plt.plot(list_of_year, world, 'b.-', label='World')


plt.xticks(list_of_year)
plt.xlabel('Year' , fontsize =  25)
plt.ylabel('consumer prices (annual %)' ,fontsize =  25)
plt.legend()

plt.show()

###### Answer  Q 2 : Does the CPI of Saudi Arabia is higher than the average CPI of the world over time?
>The saudi arabia from 1990 to 2008 was less than the world, but after 2008 became almost closely with the world 



#### Q 3 :Does the Income level affect the CPI?

In [ ]:
# INCOME : 
#High income
#Middle income
#Low income
high_income = df_clean['consumer_prices'].loc[df_clean.country_name == 'High income']
middle_income = df_clean['consumer_prices'].loc[df_clean.country_name == 'Middle income']
low_income = df_clean['consumer_prices'].loc[df_clean.country_name == 'Low income']

plt.figure(figsize=(20,10))
plt.title('Consumer Price Index (CPI) over Time ',fontdict={'fontweight':'bold', 'fontsize': 25})


plt.plot(list_of_year, high_income, 'r.-', label='High income')
plt.plot(list_of_year, middle_income, 'b.-', label='Middle income')
plt.plot(list_of_year, low_income, 'y.-' , label='Low income')

plt.xticks(list_of_year)

plt.xlabel('Year' , fontsize =  25)
plt.ylabel('consumer prices (annual %)' ,fontsize =  25)
plt.legend()

plt.show()


###### Answer  Q 3 : Does the Income level affect the CPI?

>Income : Net profit, or money that remains after expenses are subtracted from revenue.
>
>Revenue :Proceeds from the sales of products and services to customers, as well as other activities like investment.
>


>whenever higher the level of income then CPI decreasing and become more stable
>
>1- High income , low CPI
>
>2- Middle income , Moderate CPI
>
>3- Low income , Hight CPI
>
>
https://www.diffen.com/difference/Income_vs_Revenue

#### Q 4 :What are the top ten countries in terms of CPI?

In [ ]:
largest = df_clean.groupby('country_name')['consumer_prices'].mean().nlargest(10)
largest_value = largest.to_list()
largest_index = largest.index.to_list()
largest_value.reverse()
largest_index.reverse()

In [ ]:
largest_value

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

values = largest_value
y_pos = np.arange(len(values))
index = largest_index


plt.bar(y_pos, index, align='center', alpha=0.5)
plt.xticks(y_pos)
plt.ylabel('Country')
plt.title('Highest countries')

plt.show()

###### Answer  Q 4: What are the top ten countries in terms of CPI?
>1. CONGO  DEM. REP
>2. ANGOLA
3. UKRAINE
4. BRAZIL
5. BELARUS
6. ARMENIA
7. MOLDOVA
8. AZERBAIJA
9. KAZAKHSTAN
10. CROATIA



#### Q 5 :What are the least ten countries in terms of CPI?

In [ ]:
smallest = df_clean.groupby('country_name')['consumer_prices'].mean().nsmallest(10)
smallest_value = smallest.to_list()
smallest_index = smallest.index.to_list()
smallest_value.reverse()
smallest_index.reverse()

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

values = smallest_value
y_pos = np.arange(len(values))
index = smallest_index


plt.bar(y_pos, index, align='center', alpha=0.5)
plt.xticks(y_pos)
plt.ylabel('Country')
plt.title('least countries')

plt.show()

###### Answer  Q 5: What are the least ten countries in terms of CPI?
>1.	JAPAN
2.	CAYMAN ISLANDS
3.	BRUNEI DARUSSALAM
4.	SWITZERLAND
5.	ZIMBABWE 
6.	NEW CALEDONIA
7.	BAHRAIN
8.	FRANCE
9.	BOSNIA AND HERZEGOVINA
10.	FINLAND




In [ ]:
df_clean.to_excel('CPI_Dataset.xlsx')